# How to read data from BigQuery

This notebook demonstrates two ways to use BigQuery with Python
1. by using SQL via [pandas-gbq](https://pandas-gbq.readthedocs.io/en/latest/)
2. by using only Python code to extract the data of interest from BigQuery via [Ibis](https://docs.ibis-project.org/)

## Setup

First, be sure to run notebook **`Python environment setup`** in this workspace.

Then in this section we:

1. load the needed python packages
2. set the project id of the cloud project to bill for queries to BigQuery

In [1]:
import pandas as pd
import pandas_gbq
import ibis
import os

/home/jupyter-user/.local/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/jupyter-user/.local/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']

## Option 1: Retrieve filtered data from BigQuery using SQL.

The following SQL will read a subset of columns and subset of rows from a BigQuery table into a Pandas dataframe.
* [Pandas](http://pandas.pydata.org/pandas-docs/stable/) is a popular Python package for data manipulation.
* To learn more about SQL syntax see the [BigQuery standard SQL reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/).

In [3]:
sample_info = pd.read_gbq("""
SELECT
  Sample,
  Gender,
  Relationship,
  Population,
  Population_Description,
  Super_Population,
  Super_Population_Description,
  Total_Exome_Sequence,
  Main_Project_E_Platform,
  Main_Project_E_Centers
FROM
  `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`
WHERE
  -- Only include information for samples in phase 1.
  In_Phase1_Integrated_Variant_Set = TRUE
""",
    project_id=BILLING_PROJECT_ID,
    dialect='standard')

Downloading: 100%|██████████| 1092/1092 [00:00<00:00, 5283.20rows/s]


In [4]:
sample_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092 entries, 0 to 1091
Data columns (total 10 columns):
Sample                          1092 non-null object
Gender                          1092 non-null object
Relationship                    1092 non-null object
Population                      1092 non-null object
Population_Description          1092 non-null object
Super_Population                1092 non-null object
Super_Population_Description    1092 non-null object
Total_Exome_Sequence            1069 non-null float64
Main_Project_E_Platform         1092 non-null object
Main_Project_E_Centers          1092 non-null object
dtypes: float64(1), object(9)
memory usage: 85.4+ KB


In [5]:
sample_info.describe()

,Total_Exome_Sequence
count,1.069000e+03
mean,1.028243e+10
std,3.845551e+09
min,2.477265e+09
25%,8.078811e+09
50%,9.730942e+09
75%,1.142406e+10
max,3.511709e+10


In [6]:
sample_info.head()

,Sample,Gender,Relationship,Population,Population_Description,Super_Population,Super_Population_Description,Total_Exome_Sequence,Main_Project_E_Platform,Main_Project_E_Centers
0,HG00377,female,,FIN,Finnish in Finland,EUR,European,4.105720e+09,ILLUMINA,BI
1,HG01278,female,mother,CLM,"Colombian in Medellin, Colombia",AMR,American,7.717205e+09,ILLUMINA,BCM
2,NA18527,female,,CHB,"Han Chinese in Bejing, China",EAS,East Asian,5.811252e+09,ILLUMINA,BCM
3,NA19672,female,mother,MXL,"Mexican Ancestry in Los Angeles, California",AMR,American,NaN,,
4,NA20363,female,mother,ASW,African Ancestry in Southwest US,AFR,African,NaN,,


## Option 2: Retrieve filtered data from BigQuery using Python.

The following Python code will read a BigQuery table into a Pandas dataframe.

From https://cloud.google.com/community/tutorials/bigquery-ibis

*[Ibis](http://ibis-project.org/) is a Python library for doing data analysis. It offers a Pandas-like environment for executing data analysis in big data processing systems such as Google BigQuery. Ibis's primary goals are to be a type safe, expressive, composable, and familiar replacement for SQL.*

In [7]:
conn = ibis.bigquery.connect(
    project_id=BILLING_PROJECT_ID,
    dataset_id='bigquery-public-data.human_genome_variants')

In [8]:
print('ibis version: %s' % ibis.__version__)

ibis version: 1.2.0


In [9]:
sample_info_tbl = conn.table('1000_genomes_sample_info')
sample_info_tbl

BigQueryTable[table]
  name: bigquery-public-data.human_genome_variants.1000_genomes_sample_info
  schema:
    Sample : string
    Family_ID : string
    Population : string
    Population_Description : string
    Gender : string
    Relationship : string
    Unexpected_Parent_Child : string
    Non_Paternity : string
    Siblings : string
    Grandparents : string
    Avuncular : string
    Half_Siblings : string
    Unknown_Second_Order : string
    Third_Order : string
    In_Low_Coverage_Pilot : boolean
    LC_Pilot_Platforms : string
    LC_Pilot_Centers : string
    In_High_Coverage_Pilot : boolean
    HC_Pilot_Platforms : string
    HC_Pilot_Centers : string
    In_Exon_Targetted_Pilot : boolean
    ET_Pilot_Platforms : string
    ET_Pilot_Centers : string
    Has_Sequence_in_Phase1 : boolean
    Phase1_LC_Platform : string
    Phase1_LC_Centers : string
    Phase1_E_Platform : string
    Phase1_E_Centers : string
    In_Phase1_Integrated_Variant_Set : boolean
    Has_Phase1_chr

In [10]:
# Define the filter criteria.
phase_1_only = sample_info_tbl.In_Phase1_Integrated_Variant_Set == True

# Apply the filter and choose the columns to return.
phase_1_sample_info_tbl = sample_info_tbl.filter(phase_1_only)['Sample', 'Gender', 'Relationship', 'Population', 'Population_Description',
                'Super_Population', 'Super_Population_Description', 'Total_Exome_Sequence',
                'Main_Project_E_Platform', 'Main_Project_E_Centers']

In [11]:
# Optional: take a look at the SQL.
print(phase_1_sample_info_tbl.compile())

SELECT `Sample`, `Gender`, `Relationship`, `Population`,
       `Population_Description`, `Super_Population`,
       `Super_Population_Description`, `Total_Exome_Sequence`,
       `Main_Project_E_Platform`, `Main_Project_E_Centers`
FROM (
  SELECT *
  FROM `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`
  WHERE `In_Phase1_Integrated_Variant_Set` = TRUE
) t0


In [12]:
# Optional: See how much data this will return.
phase_1_sample_info_tbl.count().execute()

1092

In [13]:
# Go ahead and retrieve the data.
phase_1_sample_info_df = phase_1_sample_info_tbl.limit(1000000).execute()
phase_1_sample_info_df.shape

(1092, 10)

In [14]:
phase_1_sample_info_df.head()

,Sample,Gender,Relationship,Population,Population_Description,Super_Population,Super_Population_Description,Total_Exome_Sequence,Main_Project_E_Platform,Main_Project_E_Centers
0,HG00377,female,,FIN,Finnish in Finland,EUR,European,4.105720e+09,ILLUMINA,BI
1,HG01278,female,mother,CLM,"Colombian in Medellin, Colombia",AMR,American,7.717205e+09,ILLUMINA,BCM
2,NA18527,female,,CHB,"Han Chinese in Bejing, China",EAS,East Asian,5.811252e+09,ILLUMINA,BCM
3,NA19672,female,mother,MXL,"Mexican Ancestry in Los Angeles, California",AMR,American,NaN,,
4,NA20363,female,mother,ASW,African Ancestry in Southwest US,AFR,African,NaN,,


# Provenance

In [15]:
import datetime
print(datetime.datetime.now())

2020-01-13 21:00:09.247258


In [16]:
!pip3 freeze

google-api-core==1.15.0
google-auth-oauthlib==0.4.1
google-cloud-bigquery==1.23.1
google-cloud-core==1.1.0
ibis-framework==1.2.0
multipledispatch==0.6.0
oauthlib==3.1.0
pandas==0.25.3
pandas-gbq==0.13.0
pydata-google-auth==0.2.1
regex==2020.1.8
requests-oauthlib==1.3.0
six==1.13.0
toolz==0.10.0


Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.